In [3]:
import PySimpleGUI as pg
import yfinance as yf
import pandas as pd
import numpy as np
import math
import statistics
import requests
import csv
import datetime as dt
import openpyxl
from io import StringIO

################################################
# questionnaire de base visuel dans une fonction
def questionnaire_base():

	# theme questionnaire
	pg.theme('DarkGrey4')

	# créer les champs
	champs = [
		[pg.Text("Entrez l'information qui suit:")],
        [pg.Text('Prenom', size=(15, 1)), pg.InputText(key='prenom')],
        [pg.Text('Nom', size=(15, 1)), pg.InputText(key='nom')],
        [pg.Text('Age', size=(15, 1)), pg.InputText(key='age')],
        [pg.Button('Sauvegarder'), pg.Button('Terminer')]
	]

	# créer la fenêtre
	fenetre = pg.Window('Informations principales', champs)

	# créer un dictionnaire pour sauvegarder les informations
	informations_principales = {}

	# event loop
	while True:
		event, values = fenetre.read()

		# bouton 'quitter'
		if event in (pg.WINDOW_CLOSED, 'Terminer'):
			confirmer = pg.popup('Avez-vous enregistré avant de quitter?', title='Confirmation', button_type=pg.POPUP_BUTTONS_YES_NO, custom_text = ('Oui', 'Non'))
			if confirmer == 'Oui':
				break
			else:
				continue

		# bouton 'sauvegarder'
		if event == 'Sauvegarder':

			# vérifier le type de données
			if not values['age'].isnumeric():
				pg.PopupError("Veuillez entrer un nombre entier pour l'âge.")
				continue

			# insérer les valeurs dans le dictionnaire			
			informations_principales = values

	fenetre.close()

	return informations_principales

# utiliser le dictionnaire à l'extérieur de la fonction
informations_principales = questionnaire_base()

################################################
# dictionnaires pour les questions
# q1
q1 = "Dans quelle mesure êtes-vous préoccupé par l'inflation (le risque que votre argent achète moins de biens et services à l'avenir en raison de la hausse des prix ?"
q1_dict = {
    'a' : "Mon objectif est de minimiser les fluctuations de la valeur de mon portefeuille, même si la croissance ne suit pas le rythme de l'inflation.",
    'b' : "Mon objectif est que la croissance suive au moins le rythme de l'inflation, avec le risque de légères fluctuations de la valeur de mon portefeuille.",
    'c' : "Mon objectif est que la croissance dépasse l'inflation, avec un risque de fluctuations un peu plus importantes de la valeur de mon portefeuille.",
    'd' : "Mon objectif est que la croissance dépasse largement l'inflation, avec le risque de fluctuations importantes de la valeur de mon portefeuille."
}

# q2
q2 = "Quelle affirmation décrit le mieux votre philosophie d'investissement personnelle ?"
q2_dict = {
    'a' : "Revenu : je préfère les investissements qui peuvent générer des rendements plus constants (mais probablement plus faibles) d'année en année, en mettant l'accent principalement sur la génération de revenus. Je préfère un faible niveau de fluctuations et un faible risque de baisse au fil du temps.",
    'b' : "Croissance et revenu : je préfère les investissements qui équilibrent mes objectifs de croissance avec mes besoins de revenu. Je préfère une quantité modeste de fluctuations et un risque de baisse au fil du temps.",
    'c' : "Croissance : je suis prêt à détenir des investissements présentant un degré de fluctuations et un risque de baisse plus élevés en échange de la possibilité d'obtenir des rendements moyens plus élevés au fil du temps."
}

# q3
q3 = "Dans quelle mesure êtes-vous à l'aise avec les fluctuations potentielles de votre portefeuille ? »"
q3_dict = {
    'a' : "Des fluctuations plus faibles, un potentiel de rendement inférieur.",
    'b' : "Fluctuations modérées, potentiel de rendement modéré.",
    'c' : "Des fluctuations plus importantes, un potentiel de rendement plus élevé."
}

# q4
q4 = "Quelle affirmation décrit le mieux ce que vous pensez du compromis entre les rendements potentiels et les baisses ?"
q4_dict = {
    'a' : "Je me concentre davantage sur les baisses potentielles de la valeur de mon portefeuille. Le rendement que j'obtiens est d'une importance secondaire.",
    'b' : "Le potentiel de baisse est pour moi tout aussi important que le potentiel de rendement.",
    'c' : "Je me concentre davantage sur le potentiel de rendement de mon portefeuille que sur les baisses potentielles."
}

# q5
q5 = "Il y a eu plusieurs périodes dans l'histoire au cours desquelles la valeur du marché a chuté de 25 % ou plus en un an. Si la valeur de votre portefeuille est passée de 200 000 $ \
à 150 000 $ (25 %) en un an, comment réagiriez-vous"
q5_dict = {
    'a' : "Je déplacerais mon argent vers différents investissements pour réduire le risque de nouvelles baisses, même si cela signifiait manquer une reprise potentielle.",
    'b' : "Je serais inquiet et envisagerais de me lancer dans d'autres investissements si les baisses se poursuivaient",
    'c' : "Je laisserais mon argent là où il est et continuerais selon ma stratégie à long terme.",
    'd' : "Je considérerais cela comme une opportunité et j'envisagerais d'investir davantage si j'avais l'argent disponible."
}

# q6
q6 = "Il existe un compromis entre des fluctuations potentielles plus importantes à court terme et le potentiel de croissance à long terme d'un portefeuille, comme le montre le tableau ci-dessous. \
Avec quel portefeuille seriez-vous le plus à l'aise ?"
q6_dict = {
    'a' : "Portefeuille A",
    'b' : "Portefeuille b",
    'c' : "Portefeuille c"
}

################################################
# questionnaire Edward Jones pour déterminer la tolérance au risque de l'utilisateur
def questionnaire_EJ():

	# theme questionnaire
	pg.theme('DarkGrey4')

	# créer les champs pour les questions questions
	question_1 = [
		[pg.Text(f'1/6: {q1}')],
		[pg.Radio(q1_dict['a'], '1', key='1a')],
		[pg.Radio(q1_dict['b'], '1', key='1b')],
		[pg.Radio(q1_dict['c'], '1', key='1c')],
		[pg.Radio(q1_dict['d'], '1', key='1d')]
	]

	question_2 = [
		[pg.Text(f'2/6: {q2}')],
		[pg.Radio(q2_dict['a'], '2', key='2a')],
		[pg.Radio(q2_dict['b'], '2', key='2b')],
		[pg.Radio(q2_dict['c'], '2', key='2c')]
	]

	question_3 = [
		[pg.Text(f'3/6: {q3}')],
		[pg.Radio(q3_dict['a'], '3', key='3a')],
		[pg.Radio(q3_dict['b'], '3', key='3b')],
		[pg.Radio(q3_dict['c'], '3', key='3c')],
		[pg.Image(filename='EJ_chart_1.png')]
	]

	question_4 = [
		[pg.Text(f'4/6: {q4}')],
		[pg.Radio(q4_dict['a'], '4', key='4a')],
		[pg.Radio(q4_dict['b'], '4', key='4b')],
		[pg.Radio(q4_dict['c'], '4', key='4c')]
	]

	question_5 = [
		[pg.Text(f'5/6: {q5}')],
		[pg.Radio(q5_dict['a'], '5', key='5a')],
		[pg.Radio(q5_dict['b'], '5', key='5b')],
		[pg.Radio(q5_dict['c'], '5', key='5c')],
		[pg.Radio(q5_dict['d'], '5', key='5d')]
	]

	question_6 = [
		[pg.Text(f'6/6: {q6}')],
		[pg.Radio(q6_dict['a'], '6', key='6a')],
		[pg.Radio(q6_dict['b'], '6', key='6b')],
		[pg.Radio(q6_dict['c'], '6', key='6c')],
		[pg.Image(filename='EJ_chart_2.png')]
	]

	layout = [
		[pg.Column(question_1, key='q1'), pg.Column(question_2, visible=False, key='q2'),\
		pg.Column(question_3, visible=False, key='q3'), pg.Column(question_4, visible=False, key='q4'),\
		pg.Column(question_5, visible=False, key='q5'), pg.Column(question_6, visible=False, key='q6')],
		[pg.Button('Retour'), pg.Button('Suivant')],
    	[pg.Button('Sauvegarder'), pg.Button('Terminer')]
	]

	# créer la fenêtre
	fenetre = pg.Window('Questionnaire Edward Jones', layout)

	data_EJ = {}
	question_nb = 1

	# event loop
	while True:
		event, values = fenetre.read()

		# bouton 'quitter'
		if event in (pg.WINDOW_CLOSED, 'Terminer'):
			confirmer = pg.popup('Avez-vous enregistré avant de quitter?', title='Confirmation', button_type=pg.POPUP_BUTTONS_YES_NO, custom_text = ('Oui', 'Non'))
			if confirmer == 'Oui':
				break
			else:
				continue

		# bouton 'Sauvegarder'
		if event == 'Sauvegarder':
			data_EJ = values

		# bouton 'suivant'
		if event == 'Suivant':
			if question_nb == 6:
				fenetre[f'q{question_nb}'].update(visible=True)
			else:
				fenetre[f'q{question_nb}'].update(visible=False)
				question_nb += 1
				fenetre[f'q{question_nb}'].update(visible=True)

		# bouton 'retour'
		if event == 'Retour':
			if question_nb == 1:
					fenetre[f'q{question_nb}'].update(visible=True)
			else:
				fenetre[f'q{question_nb}'].update(visible=False)
				question_nb -= 1
				fenetre[f'q{question_nb}'].update(visible=True)

	fenetre.close()

	return data_EJ

data_EJ = questionnaire_EJ()

################################################
# attributions des points pour déterminer la tolérance au risque
reponses_vrai = []

for key, value in data_EJ.items():
	if value == True:
		reponses_vrai.append(key)

points_tolerance_risque = 0

for reponse in reponses_vrai:

	# q1
	if reponse == '1b':
		points_tolerance_risque += 5
	elif reponse == '1c':
		points_tolerance_risque += 12
	elif reponse == '1d':
		points_tolerance_risque += 17

	# q2
	if reponse == '2b':
		points_tolerance_risque += 8
	elif reponse == '2c':
		points_tolerance_risque += 16

	# q3
	if reponse == '3b':
		points_tolerance_risque += 8
	elif reponse == '3c':
		points_tolerance_risque += 16

	# q4
	if reponse == '4b':
		points_tolerance_risque += 8
	elif reponse == '4c':
		points_tolerance_risque += 17

	# q5
	if reponse == '5b':
		points_tolerance_risque += 6
	elif reponse == '5c':
		points_tolerance_risque += 12
	elif reponse == '5d':
		points_tolerance_risque += 17

	# q6
	if reponse == '6b':
		points_tolerance_risque += 8
	elif reponse == '6c':
		points_tolerance_risque += 17

# déterminer tolérance au risque
if (points_tolerance_risque <= 18):
    tolerance_risque = 'bas'
elif (19 <= points_tolerance_risque <= 39):
    tolerance_risque = 'bas_moyen'
elif (40 <= points_tolerance_risque <= 59):
    tolerance_risque = 'moyen'
elif (60 <= points_tolerance_risque <= 79):
    tolerance_risque = 'moyen_haut'
elif (80 <= points_tolerance_risque <= 100):
    tolerance_risque = 'haut'

### questionnaire spécifique au portefeuille
def questionnaire_pf():

	# theme questionnaire
	pg.theme('DarkGrey4')

	# créer les champs
	champs = [
		[pg.Text("Entrez l'information qui suit:")],
        [pg.Text('Montant à Investir', size=(25, 1)), pg.InputText(key='montant')],
        [pg.Text('Horizon de Placement (années)', size=(25, 1)), pg.InputText(key='horizon')],
        [pg.Text()],
        [pg.Text('Voulez-vous faire une mise-à-jour des données?\n(Peut prendre quelques minutes.)')],
        [pg.Radio('Oui', '1', key='MAJ_oui'), pg.Radio('Non', '1', key='MAJ_non')],
        [pg.Button('Sauvegarder'), pg.Button('Terminer')]
	]

	# créer la fenêtre
	fenetre = pg.Window('Questionnaire sur le Portefeuille', champs)

	# créer un dictionnaire pour sauvegarder les informations
	informations_portefeuille = {}

	# event loop
	while True:
		event, values = fenetre.read()

		# bouton 'quitter'
		if event in (pg.WINDOW_CLOSED, 'Terminer'):
			confirmer = pg.popup('Avez-vous enregistré avant de quitter?', title='Confirmation', button_type=pg.POPUP_BUTTONS_YES_NO, custom_text = ('Oui', 'Non'))
			if confirmer == 'Non':
				continue
			else:
				pg.popup_ok("Veuillez patienter, le téléchargement des données ainsi que les calculs peuvent prendre quelques minutes.\
					\n\nClickez sur 'OK' puis une nouvelle fenêtre apparaitra lorsque les résultats seront prêts.", title="Confirmation")
				break
			

		# bouton 'sauvegarder'
		if event == 'Sauvegarder':

			# vérifier le type de données
			if not values['montant'].isnumeric():
				pg.PopupError("Veuillez entrer un nombre entier pour le montant à investir.")
				continue

			if not values['horizon'].isnumeric():
				pg.PopupError("Veuillez entrer un nombre entier pour l'horizon de placement.")
				continue

			# insérer les valeurs dans le dictionnaire			
			informations_portefeuille = values

	fenetre.close()

	return informations_portefeuille

# utiliser le dictionnaire à l'extérieur de la fonction
informations_portefeuille = questionnaire_pf()



### importer le document .csv avec les tickers sur le sp500
# ouvrir le .csv
sp500_df = pd.read_csv('sp500.csv')

# append les tickers dans une liste
ticker_list = []

for ticker in sp500_df['Symbol']:
    ticker_list.append(ticker)

# append les secteurs dans une liste
sector_list = []

for sector in sp500_df['Sector']:
    sector_list.append(sector)

# créer un dictionnaire avec les tickers et secteurs
ticker_sector_pair = zip(ticker_list, sector_list)
ticker_sector_dict = dict(ticker_sector_pair)


### télécharger les données de tous les tickers
# prendre la date d'aujourd'hui
today_date = dt.datetime.today().strftime('%Y-%m-%d')

# créer une fonction qui télécharge les données mensuelles d'un ticker
def get_ticker_data_m(ticker_list, min_date='2023-11-30'):
    for ticker in ticker_list:
        try:
            data = yf.download(ticker.upper(), '2015-01-01', today_date, interval="1mo")

            # ajouter monthly returns
            data['Monthly_Returns'] = data['Adj Close'].pct_change()

            last_date = str(data.index.max())
            last_date_reformat = dt.date(int(last_date[0:4]), int(last_date[5:7]), int(last_date[8:10]))
            min_date_reformat = dt.date(int(min_date[0:4]), int(min_date[5:7]), int(min_date[8:10]))

            # vérifier si les données sont disponibles 
            if last_date_reformat < min_date_reformat:
                print(f'{ticker.upper()} - Pas de données recentes')
            else:
                # available_tickers.append(ticker)
                data.to_csv(f'ticker_data/m/{ticker.upper()}.csv')
                print(f'{ticker.upper()} - Données recentes disponibles et téléchargées')

        except:
            print("erreur telechargement")
            continue
        
# créer une fonction qui télécharge les données quotidiennes d'un ticker 
def get_ticker_data_d(ticker_list, min_date='2023-11-30'):
    for ticker in ticker_list:
        try:
            data = yf.download(ticker.upper(), '2015-01-01', today_date)

            # ajouter daily returns
            data['Daily_Returns'] = data['Adj Close'].pct_change()

            last_date = str(data.index.max())
            last_date_reformat = dt.date(int(last_date[0:4]), int(last_date[5:7]), int(last_date[8:10]))
            min_date_reformat = dt.date(int(min_date[0:4]), int(min_date[5:7]), int(min_date[8:10]))

            # vérifier si les données sont disponibles 
            if last_date_reformat < min_date_reformat:
                print(f'{ticker.upper()} - Pas de données recentes')
            else:
                # available_tickers.append(ticker)
                data.to_csv(f'ticker_data/d/{ticker.upper()}.csv')
                print(f'{ticker.upper()} - Données recentes disponibles et téléchargées')

        except:
            print("erreur telechargement")
            continue

# créer une fonction qui télécharge les données du SPY (sp500 utilisé comme index du marché)
def sp500_data():
    data = yf.download('SPY', '2015-01-01', today_date, interval="1mo")

    # ajouter monthly returns
    data['Monthly_Returns'] = data['Adj Close'].pct_change()

    data.to_csv('SPY.csv')
    
if informations_portefeuille['MAJ_oui'] == True:
    get_ticker_data_m(ticker_list)
    get_ticker_data_d(ticker_list)

    sp500_data()
else:
    print('Pas de mise à jour')
    

    
# obtenir le taux sans risque (CORRA) le plus recent
def corra():
    corra_data = requests.get('https://www.bankofcanada.ca/valet/observations/group/CORRA/csv')
    
    data = []

    if corra_data.status_code == 200:
        # Use csv.reader to handle CSV parsing
        reader = csv.reader(StringIO(corra_data.text))
        
        # Extract the second column from each row
        for row in reader:
            # Ensure the row has at least 2 columns before trying to access the second column
            if len(row) >= 2:
                data.append(row[1])
            #print(row)
    else:
        print('erreur')
    
    return float(data[len(data)-1])
    # ajouter des données hors ligne comme back up

def bond_yield(horizon):
    bond_data = requests.get('https://www.bankofcanada.ca/valet/observations/group/bond_yields_marketable/csv')
    
    data = []
    
    if bond_data.status_code == 200:
        reader = csv.reader(StringIO(bond_data.text))
        
        for row in reader:
            if len(row) == 5:
                if 1 <= horizon < 3:
                    data.append(row[1])
                elif 3 <= horizon < 5:
                    data.append(row[2])
                elif 5 <= horizon < 10:
                    data.append(row[3])
                elif 10 <= horizon:
                    data.append(row[4])
    else:
        print('erreur')
        
    return float(data[len(data)-1])

# créer une fonction qui retourne la dernière journée disponible pour un ticker
def ticker_last_day(ticker):
	ticker_df = pd.read_csv(f'ticker_data/d/{ticker.upper()}.csv')
	dates = ticker_df['Date']
	ticker_last_date = dates.iloc[len(dates)-1]

	return ticker_last_date

# estimer les rendement d'une action pour la prochaine année avec une simulation monte carlo
def ticker_rate_sim(ticker, no_simulations=10000):
	ticker_df = pd.read_csv(f'ticker_data/d/{ticker.upper()}.csv')

	last_date = ticker_last_day(ticker)
	start_date = (dt.datetime.strptime(last_date, "%Y-%m-%d") - dt.timedelta(days=365)).strftime("%Y-%m-%d")

	daily_returns = ticker_df['Daily_Returns'].dropna()

	mean_return = daily_returns.mean()
	volatility = daily_returns.std()

	simulations = np.zeros((no_simulations, 252))

	for i in range(no_simulations):
		normal_random_returns = np.random.normal(mean_return, volatility, 252)

		cumulative_returns = np.cumprod(1 + normal_random_returns) - 1

		simulations[i, :] = cumulative_returns
	
	median_returns = np.median(simulations[:, -1])

	return median_returns





### analyse des données
### écart type
# créer un dictionnaire pour enregistrer les écarts types des tickers
ticker_stdev_dict = {}

# créer une fonction qui calcul l'écart type d'un ticker et l'enregistre dans un dictionnaire
def ticker_stdev(ticker):
	ticker_df = pd.read_csv(f'ticker_data/m/{ticker.upper()}.csv')

	stdev = np.std(ticker_df['Monthly_Returns'])

	ticker_stdev_dict[ticker.upper()] = stdev 

# si les données sont disponibles, calculer l'écart type pour chaque ticker
for ticker in ticker_list:
    try:
        ticker_stdev(ticker)
    except:
        continue
        
# calculer la moyenne d'ecart type
stdev_list = []
bad_stdev = []
for key, value in ticker_stdev_dict.items():
    if math.isnan(value):
        bad_stdev.append(key)
    else:
        stdev_list.append(value)

for item in bad_stdev:
    del ticker_stdev_dict[item]
    
mean_stdev = np.mean(stdev_list)
#print(mean_stdev)

### beta
def beta(ticker):
    ticker_csv = pd.read_csv(f'ticker_data/m/{ticker.upper()}.csv')
    ticker_returns = ticker_csv['Monthly_Returns'].dropna()
    
    market_csv = pd.read_csv('SPY.csv')
    market_returns = market_csv['Monthly_Returns'].dropna()
    
    covariance = np.cov(ticker_returns, market_returns)[0, 1]
    market_variance = np.var(market_returns)
    
    beta = covariance / market_variance
    
    return beta

beta_dict = {}
for ticker in ticker_list:
    try:
        beta_dict[ticker] = beta(ticker)
    except:
        continue

# sort stocks per beta
low_beta_stock = []
high_beta_stock = []
medium_beta_stock = []

def beta_sort(beta_dict, high=1.1, low=0.9):
	for key, value in beta_dict.items():
	    if value >= high:
	        high_beta_stock.append(key)
	    elif value <= low:
	    	low_beta_stock.append(key)
	    else:
	   		medium_beta_stock.append(key)

beta_sort(beta_dict)

# créer un sélecteur de ticker
def selecteur_ticker(tolerance_risque, nb_tickers=20, sort_volume=True):
	if tolerance_risque == 'moyen':
		niveau_1 = low_beta_stock
	elif tolerance_risque == 'moyen_haut':
		niveau_1 = medium_beta_stock
	elif tolerance_risque == 'haut':
		niveau_1 = high_beta_stock

	if sort_volume == True:
		ticker_volume_dict = {}

		for ticker in niveau_1:
			ticker_csv = pd.read_csv(f'ticker_data/m/{ticker.upper()}.csv')
			last_row = ticker_csv.iloc[len(ticker_csv)-1]
			volume = last_row['Volume']
			ticker_volume_dict[ticker] = volume

		sorted_items = sorted(ticker_volume_dict.items(), key=lambda x: x[1], reverse=True)

		# Convert the sorted items back to a dictionary
		sorted_dict = dict(sorted_items)
		sorted_list = []
		for key, value in sorted_dict.items():
			sorted_list.append(key)

	tickers_choisi = []

	# prendre les x stock avec le plus grand volume
	# vérifier que le nombre de tickers ne dépasse pas le nombre de tickers disponibles
	tickers_choisi = sorted_list[0 : nb_tickers]

	return tickers_choisi

# créer une fonction qui calcule le taux de rendement estimé des actions dans le portefeuille
def pf_ticker_rate(ticker_list):
	ticker_rates = []

	for ticker in ticker_list:
		ticker_rates.append(ticker_rate_sim(ticker, no_simulations=20000))

	r_moyen = statistics.mean(ticker_rates)

	return r_moyen

### créer les 5 portfolios avec risque appropriés
## create a portfolio chooser function that takes into account ability to take risk and horizon

# 1 100% CORRA
# 2 50% CORRA 50% bonds
# 3 50% CORRA 25% bonds 25% stocks (b < 1)
# 4 25% CORRA 25% bonds 50% stocks (0.9 <= b <= 1.1)
# 5 25% bonds 75% stocks (1.1 <= b)
ratios_portefeuille = {}

if tolerance_risque == 'bas':
	ratios_portefeuille = {
	'ratio_corra' : 1
	}
elif tolerance_risque == 'bas_moyen':
	ratios_portefeuille = {
	'ratio_corra' : 0.5,
	'ratio_bonds' : 0.5
	}
elif tolerance_risque == 'moyen':
	ratios_portefeuille = {
	'ratio_corra' : 0.5,
	'ratio_bonds' : 0.25,
	'ratio_stocks' : 0.25
	}
elif tolerance_risque == 'moyen_haut':
	ratios_portefeuille = {
	'ratio_corra' : 0.25,
	'ratio_bonds' : 0.25,
	'ratio_stocks' : 0.50
	}
elif tolerance_risque == 'haut':
	ratios_portefeuille = {
	'ratio_bonds' : 0.25,
	'ratio_stocks' : 0.75
	}


def portfolio_yearly_return(montant, horizon, tolerance_risque):
	if tolerance_risque == 'bas':
		r_corra = (corra()/100) * ratios_portefeuille['ratio_corra']
		r = r_corra

	elif tolerance_risque == 'bas_moyen':
		r_corra = (corra()/100) * ratios_portefeuille['ratio_corra']
		r_bonds = (bond_yield(horizon)/100) * ratios_portefeuille['ratio_bonds']
		r = r_corra + r_bonds

	elif tolerance_risque == 'moyen':
		r_corra = (corra()/100) * ratios_portefeuille['ratio_corra']
		r_bonds = (bond_yield(horizon)/100) * ratios_portefeuille['ratio_bonds']
		r_stocks = (pf_ticker_rate(selecteur_ticker(tolerance_risque))) * ratios_portefeuille['ratio_stocks']
		r = r_corra + r_bonds + r_stocks

	elif tolerance_risque == 'moyen_haut':
		r_corra = (corra()/100) * ratios_portefeuille['ratio_corra']
		r_bonds = (bond_yield(horizon)/100) * ratios_portefeuille['ratio_bonds']
		r_stocks = (pf_ticker_rate(selecteur_ticker(tolerance_risque))) * ratios_portefeuille['ratio_stocks']
		r = r_corra + r_bonds + r_stocks

	elif tolerance_risque == 'haut':
		r_bonds = (bond_yield(horizon)/100) * ratios_portefeuille['ratio_bonds']
		r_stocks = (pf_ticker_rate(selecteur_ticker(tolerance_risque))) * ratios_portefeuille['ratio_stocks']
		r = r_bonds + r_stocks

	fv = montant * (1 + r) ** horizon

	pf_results = {
		'taux' : r,
		'fv' : fv
	}

	return pf_results

pf_yearly_returns = portfolio_yearly_return(float(informations_portefeuille['montant']), float(informations_portefeuille['horizon']), tolerance_risque)
print(pf_yearly_returns)


#bond rate
bond_yield(float(informations_portefeuille['horizon']))
#stock names
selecteur_ticker(tolerance_risque)
#stock qty


def excel_insert(variable, case, feuille='data', fichier_excel='rapport.xlsx'):
    """
    Insère la valeur de la variable dans la case spécifiée de la feuille Excel.

    :param variable: La valeur à insérer dans Excel.
    :param case: La case Excel où insérer la valeur (par exemple, 'A1', 'B2', etc.).
    :param feuille: Le nom de la feuille dans laquelle insérer la valeur.
    :param fichier_excel: Le nom du fichier Excel (par défaut, 'votre_fichier.xlsx').
    """
    try:
        # Charge le classeur Excel
        classeur = openpyxl.load_workbook(fichier_excel)

        # Sélectionne la feuille spécifiée
        feuille_active = classeur[feuille]

        # Insère la valeur dans la case spécifiée
        feuille_active[case] = variable

        # Enregistre les modifications dans le fichier Excel
        classeur.save(fichier_excel)
        print(f"La valeur {variable} a été insérée avec succès dans la case {case} de la feuille {feuille}.")
        
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")

#prenom
excel_insert(informations_principales['prenom'], 'B1')
#nom
excel_insert(informations_principales['nom'], 'B2')
#age
excel_insert(informations_principales['age'], 'B3')
#montant
excel_insert(float(informations_portefeuille['montant']), 'B4')
#horizon
excel_insert(float(informations_portefeuille['horizon']), 'B5')
#tolerance_risque
excel_insert(tolerance_risque, 'B6')
# taux pf
excel_insert(pf_yearly_returns['taux'], 'B7')
#composition pf
print(ratios_portefeuille)
for key, value in ratios_portefeuille.items():
    if key == 'ratio_corra':
        excel_insert(value, 'E2')
    if key == 'ratio_bonds':
        excel_insert(value, 'E3')
    if key == 'ratio_stocks':
        excel_insert(value, 'E4')
# dernière MaJ
ticker_last_day('AAPL')
# CORRA
excel_insert(corra() / 100, 'B9')
# bond name
# bond rate
excel_insert(bond_yield(float(informations_portefeuille['horizon'])) / 100, 'B10')
# stock yield
excel_insert((pf_ticker_rate(selecteur_ticker(tolerance_risque))), 'B11')


KeyError: 'MAJ_oui'